### 1. 관련 모듈 설치하기
-----

In [ ]:
!pip install gradio

### 2. 캡차 훈련 데이터를 수동으로 다운로드
-----

In [ ]:
import requests
import os

from PIL import Image
from io import BytesIO
import numpy as np

# HTTP 요청에 대한 통합된 인터페이스를 제공하기 위한 라이브러리
class Request :
  # 자연스러운 요청을 위한 기반 헤더
  GLOBAL_HEADERS = {"Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                    "Accept-Encoding" : "gzip, deflate, br",
                    "Accept-Language" : "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}

  # HTTP GET 요청을 수행하고 응답을 받기 위해서
  @staticmethod
  def get(url, headers={}, cookies={}) :
    headers.update(Request.GLOBAL_HEADERS)
    return requests.get(url, headers=headers, cookies=cookies)
  
  # HTTP POST 요청을 수행해고 응답을 받기 위해서
  @staticmethod
  def post(url, headers={}, cookies={}, data={}) :
    headers.update(Request.GLOBAL_HEADERS)
    return requests.post(url, headers=headers, cookies=cookies, data=data)

# DC 인사이드의 캡차 이미지를 얻기 위해서
class Dcincide_Captcha_API :
  def __init__(self) :
    self.GALL_ID = "birthrate"
    self.KCAPTCHA_TYPE = "write"
    self.GALL_TYPE = "M"

    self.SESSION_COOKIES = Request.get(f"https://gall.dcinside.com/mgallery/board/write/?id={self.GALL_ID}").cookies
    self.SESSION_COOKIE_INFO = {"PHPSESSID":self.SESSION_COOKIES["PHPSESSID"], "ci_c":self.SESSION_COOKIES["ci_c"]}
    self._init_Captcha_Session()
  
  # 새로운 캡차 이미지를 얻기 위해서 캡차 세션을 초기화시킴
  def _init_Captcha_Session(self) :
    SESSION_HTTP_HEADER = {"Content-Type":"application/x-www-form-urlencoded; charset=UTF-8", "X-Requested-With":"XMLHttpRequest"}
    SESSION_DATAS = {"ci_t":self.SESSION_COOKIES["ci_c"], "gall_id":self.GALL_ID, "kcaptcha_type":self.KCAPTCHA_TYPE, "_GALLTYPE_":self.GALL_TYPE}
    Request.post("https://gall.dcinside.com/kcaptcha/session", SESSION_HTTP_HEADER, self.SESSION_COOKIE_INFO, SESSION_DATAS)

  # 생성된 캡차 세션에 존재하는 캡차 이미지를 PIL 형태로 얻기 위해서
  def _captcha_Image(self) :
    IMG_RES = Request.get(f"https://gall.dcinside.com/kcaptcha/image/?gall_id={self.GALL_ID}&kcaptcha_type={self.KCAPTCHA_TYPE}", {}, self.SESSION_COOKIE_INFO)
    return Image.open(BytesIO(IMG_RES.content))

  # 새로운 캡차 문자열 이미지를 얻기 위해서
  def new_Captcha_Image(self) :
    self._init_Captcha_Session()
    return self._captcha_Image()
  
  # 이전과 정답이 동일하면서 형태만 다른 캡차 문자열 이미지를 얻기 위해서
  def same_Captcha_Image(self) :
    return self._captcha_Image()

# 파일시스템 관련 조작을 위해서
class File_System :
  # 디렉토리가 없을 경우 생성시키기 위해서
  @staticmethod
  def make_Directory_If_Not_Exist(dir_path) :
    if not os.path.exists(dir_path) : os.mkdir(dir_path)
  
  # 디렉토리 내부의 파일 개수를 반환받기 위해서
  @staticmethod
  def count_Num_Of_Files(dir_path) :
    return len(os.listdir(dir_path))

  # 특정 디렉토리를 .zip 형태로 압축시키기 위해서
  @staticmethod
  def compress_With_Zip(dir_path, zip_name) :
    os.system(f"zip -r {zip_name} {dir_path}")

  # 특정 디렉토리를 완전하게 삭제시키기 위해서
  @staticmethod
  def remove_Directory(dir_path) :
    os.system(f"rm -r {dir_path}")

# 자료형변환을 간편하게 수행하기 위한 라이브러리
class Convert :
  # 넘파이 배열을 PIL 이미지 형태로 변환시키기 위해서
  @staticmethod
  def numpy_Array_To_PIL_Image(numpy_array, color_channel="RGB") :
    return Image.fromarray(np.uint8(numpy_array)).convert(color_channel)

In [ ]:
import gradio as gr
import time

IMAGE_DIRECTORY = "./captchas" # 캡차 이미지를 저장시킬 폴더
GOAL_FILE_COUNT = 50 # 총 생성시킬 캡차 이미지 목표 개수

File_System.make_Directory_If_Not_Exist(IMAGE_DIRECTORY)
CAPTCHA_API = Dcincide_Captcha_API()
LOADED_CAPTCHA_IMAGE = CAPTCHA_API.new_Captcha_Image()


# 입력된 (캡차 이미지, 캡차 정답)을 특정 폴더에 저장시키기 위해서
def save_Captcha_Image(captcha_image_array, captcha_answer) :
  global CAPTCHA_API

  while True :
    try :
      LOADED_CAPTCHA_IMAGE = CAPTCHA_API.new_Captcha_Image()
      break
    except :
      time.sleep(3.0)
      CAPTCHA_API = Dcincide_Captcha_API()

  TOTAL_FILE_COUNT = File_System.count_Num_Of_Files(IMAGE_DIRECTORY)
  if not captcha_answer :
    return [LOADED_CAPTCHA_IMAGE, "", f"Current captcha was passed ! (progress : {GOAL_FILE_COUNT}/{TOTAL_FILE_COUNT})"]

  IMAGE_FILE_PATH_TO_SAVE = f"{IMAGE_DIRECTORY}/{captcha_answer}.jpg"
  Convert.numpy_Array_To_PIL_Image(captcha_image_array).save(IMAGE_FILE_PATH_TO_SAVE)
  if TOTAL_FILE_COUNT+1 == GOAL_FILE_COUNT :
    File_System.compress_With_Zip(IMAGE_DIRECTORY, "./" + IMAGE_DIRECTORY.split("/")[-1] + ".zip")
    raise Exception("GOAL REACHED !") 

  return [LOADED_CAPTCHA_IMAGE, "", f"Inputed image was successfully saved in '{IMAGE_FILE_PATH_TO_SAVE}' (progress : {GOAL_FILE_COUNT}/{TOTAL_FILE_COUNT+1})"]


with gr.Blocks() as base_block :
  CAPTCHA_IMAGE = gr.Image(value=LOADED_CAPTCHA_IMAGE, label="Captcha Image")
  CAPTCHA_ANSWER = gr.Textbox(label="Captcha Answer")
  RESULT_LOG = gr.Textbox(label="Save result")

  CAPTCHA_ANSWER.submit(fn=save_Captcha_Image, inputs=[CAPTCHA_IMAGE, CAPTCHA_ANSWER], outputs=[CAPTCHA_IMAGE, CAPTCHA_ANSWER, RESULT_LOG])

base_block.launch(debug=True)

### 3. 신경망을 이용한 자동화된 캡차 이미지 훈련 데이터 다운로드
-----

In [ ]:
# 관련 모듈들을 다운로드 받고, 주요 폴더를 미리 생성시키기 위해서
!pip install pytorch-lightning torchmetrics

In [ ]:
# 데이터 정보 활용을 위한 압축해제
!unzip ./Captcha_Train_Set_7000.zip
!rm ./Captcha_Train_Set_7000.zip

In [ ]:
import string
import glob
import numpy as np

from torch.utils.data.dataset import Dataset
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Normalize

# 주어진 이미지 파일 경로의 이미지들을 이용해서 캡차 데이터셋을 생성시키기 위해서
class Captcha_Dataset(Dataset) :
  def __init__(self, image_dir_path) :
    self.CORPUS = string.ascii_lowercase + string.digits
    self.BOW = self.__make_BOW(self.CORPUS)
    self.IMAGE_FILE_PATHS = glob.glob(image_dir_path + "/*.jpg")
    self.COMPOSE = Compose([
      ToTensor(),
      Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])

  def __len__(self) :
    return len(self.IMAGE_FILE_PATHS)

  def __getitem__(self, i) :
    IMAGE = self.COMPOSE(Image.open(self.IMAGE_FILE_PATHS[i]).convert("L").convert("RGB"))
    DATA = np.array(IMAGE).astype(np.float32)
    LABEL = np.array(self.__get_seq(self.__get_Label_From_Image_Path(self.IMAGE_FILE_PATHS[i])))
    return DATA, LABEL

  # 주어진 이미지 경로로부터 라벨을 추출시킨 결과를 반환하기 위해서
  def __get_Label_From_Image_Path(self, image_path) :
    return image_path.split("/")[-1].split(".")[0]

  # 주어진 문자열들을 BOW를 이용해서 정수 리스트로 변환시키기 위해서
  def __get_seq(self, letters) :
    return list(map(lambda letter : self.BOW[letter], letters))

  # 주어진 문자열들에 대한 BOW를 생성시키고 반환하기 위해서
  def __make_BOW(self, corpus) :
    bow = {"<pad>":0}

    for letter in corpus :
      if letter not in bow :
        bow[letter] = len(bow)

    return bow

In [ ]:
# 학습, 검증, 테스트 데이터셋 불러오고 관련 데이터 로더를 생성시키기 위해서
from torch.utils.data.dataloader import DataLoader

TRAIN_DATASET = Captcha_Dataset("./Captcha_Train_Set_7000/train_data")
VALID_DATASET = Captcha_Dataset("./Captcha_Train_Set_7000/valid_data")
TEST_DATASET = Captcha_Dataset("./Captcha_Train_Set_7000/test_data")

TRAIN_LOADER = DataLoader(TRAIN_DATASET, batch_size=8, shuffle=True, drop_last=True)
VALID_LOADER = DataLoader(VALID_DATASET, batch_size=8, shuffle=False, drop_last=True)
TEST_LOADER = DataLoader(TEST_DATASET, batch_size=8, shuffle=False, drop_last=True)

In [ ]:
import pytorch_lightning as pl
import torchmetrics

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.adam import Adam


# 이미지 크기를 줄이면서 ResNet 구조를 사용하기 위한 기본 모듈
class Basic_ResNet_Downsample_Layer(nn.Module) :
  def __init__(self, in_channels, out_channels) :
    super().__init__()
    
    self.CONV_1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3, 5), stride=(2, 1))
    self.CONV_2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(3, 3), padding=1)

    self.BN_1 = nn.BatchNorm2d(num_features=out_channels)
    self.BN_2 = nn.BatchNorm2d(num_features=out_channels)

    self.LERU = nn.ReLU()

    self.CONV_DOWN_SAMPLE = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3, 5), stride=(2, 1))

    self.LAYER_SEQ = nn.Sequential(
      self.CONV_1, self.BN_1, self.LERU,
      self.CONV_2, self.BN_2
    )
  
  def forward(self, x) :
    x = self.LAYER_SEQ(x) + self.CONV_DOWN_SAMPLE(x)
    x = self.LERU(x)
    return x

# 이미지 크기를 유지하면서 ResNet 구조를 사용하기 위한 기본 모듈
class Basic_ResNet_Maintain_Layer(nn.Module) :
  def __init__(self, in_channels, out_channels) :
    super().__init__()
    
    self.CONV = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3, 3), padding=1)
    self.BN = nn.BatchNorm2d(num_features=out_channels)
    self.LERU = nn.ReLU()

    self.LAYER_SEQ = nn.Sequential(
      self.CONV, self.BN, self.LERU
    )
  
  def forward(self, x) :
    return self.LAYER_SEQ(x) + x


# 셀프 어텐션을 사용하는 Skip Layer 구조를 사용하기 위한 기본 모둘
class Basic_Self_Attension_Skip_Layer(nn.Module) :
  def __init__(self, input_size) :
    super().__init__()

    self.ATTENTION = torch.nn.MultiheadAttention(input_size, 8)
    self.BN = nn.BatchNorm1d(num_features=8)
  
  def forward(self, x) :
    x_ = x
    x, _ = self.ATTENTION(x, x, x)
    x = self.BN(x)
    return x + x_


# 기본 모듈을 겹겹히 조합하기 위한 주요 신경망 모듈
class CRNN_Module(pl.LightningModule) :
  def __init__(self, bow) :
    super().__init__()
    self.BOW = bow
    self.REV_BOW = list(self.BOW.keys())

    resnet_layers = []
    resnet_layers.append(Basic_ResNet_Downsample_Layer(in_channels=3, out_channels=32))
    for layer_index in range(1, 47+1) :
      if layer_index%12 == 0 : resnet_layers.append(Basic_ResNet_Downsample_Layer(in_channels=32, out_channels=32))
      else : resnet_layers.append(Basic_ResNet_Maintain_Layer(in_channels=32, out_channels=32))
    resnet_layers.append(nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(2, 5)))
    self.RESNET_LAYER_SEQ = nn.Sequential(*resnet_layers)


    attenion_layers = [Basic_Self_Attension_Skip_Layer(32) for _ in range(5)]
    self.SELF_ATTENSION_SKIP_LAYER_SEQ = nn.Sequential(*attenion_layers)


    self.FC_1 = nn.Linear(32, 64)
    self.FC_2 = nn.Linear(64, len(self.BOW))
    self.LERU = nn.ReLU()

    self.FC_LAYER_SEQ = nn.Sequential(
        self.FC_1, self.LERU, self.FC_2
    )
  
  def forward(self, x) :
    x = self.RESNET_LAYER_SEQ(x)
    x = x.view(x.shape[0], 32, -1)
    x = x.permute(2, 0, 1)

    x = self.SELF_ATTENSION_SKIP_LAYER_SEQ(x)
    
    x = self.FC_LAYER_SEQ(x)

    x = F.log_softmax(x, dim=-1)
    return x


  def training_step(self, batch, batch_idx) :
    CTC_LOSS = self.__forward_To_CTC_Loss(batch)
    self.log('train_ctc_loss', CTC_LOSS, on_epoch=True, prog_bar=True)
    self.log('train_accuracy', self.__accuracy(batch), on_epoch=True, prog_bar=True)
    return CTC_LOSS
  
  def validation_step(self, batch, batch_idx) :
    self.log('valid_accuracy', self.__accuracy(batch), on_epoch=True, prog_bar=True)
  
  def test_step(self, batch, batch_idx) :
    self.log('test_accuracy', self.__accuracy(batch), on_epoch=True, prog_bar=True)
  
  def predict_step(self, batch, batch_idx) :
    X, Y = batch
    return self.__pred_To_Letters(X)


  def configure_optimizers(self):
      return Adam(self.parameters(), lr=1e-4)


  # 주어진 배치에 대한 CTC 손실을 반환시킥 위해서
  def __forward_To_CTC_Loss(self, batch) :
    X, Y = batch
    PREDS = self(X)

    PREDS_SIZE = torch.IntTensor([PREDS.size(0)]*PREDS.size(1))
    TARGET_SIZE = torch.IntTensor([len(y_each) for y_each in Y])

    CTC_LOSS = nn.CTCLoss(blank=0)(PREDS, Y, PREDS_SIZE, TARGET_SIZE)
    return CTC_LOSS
  
  # 주어진 배치에 대한 정확도를 반환시키기 위해서
  def __accuracy(self, batch) :
    X, Y = batch
    Y_LETTERS = self.__y_To_Letters(Y)
    PRED_LETTERS = self.__pred_To_Letters(X)

    correct_count = 0
    for pred_index in range(len(PRED_LETTERS)) :
      if PRED_LETTERS[pred_index] == Y_LETTERS[pred_index] : correct_count += 1
    
    ACCURACY = correct_count/len(PRED_LETTERS)
    return ACCURACY
  
  # 신경망에서 예측된 백터를 문자열들로 반환시키기 위해서
  def __pred_To_Letters(self, x) :
    PREDS = self(x).transpose(1,0)
    PRED_ARGMAXS = torch.argmax(PREDS, dim=-1)

    output_pred_letters = []
    for pred_argmax in PRED_ARGMAXS :
      PRED_ARGMAX_PROCESSED = self.__process_Model_Predict(pred_argmax)
      PRED_LETTERS = self.__predict_To_Letters(PRED_ARGMAX_PROCESSED)
      output_pred_letters.append(PRED_LETTERS)
    return output_pred_letters

  # CTC 손실로 중복 예측된 라벨들의 중복을 제거시키고 정제하기 위해서
  def __process_Model_Predict(self, pred_argmax) :
    pred_letters = []
    prev_letter = pred_argmax[0].item()
    if prev_letter != 0 : pred_letters.append(prev_letter)

    for letter in pred_argmax :
      if letter.item() != 0 and letter.item() != prev_letter :
        pred_letters.append(letter.item())
      prev_letter = letter.item()
    return pred_letters
  
  # 예측된 코드를 문자열로 변환시키기 위해서
  def __predict_To_Letters(self, pred) :
    return "".join([self.REV_BOW[code] for code in pred])
  
  # 정답 백터를 문자열 리스트로 변환시키기 위해서
  def __y_To_Letters(self, y) :
    return ["".join([self.REV_BOW[code] for code in y_each]) for y_each in y]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# 신경망을 통해서 캡차 데이터를 일관성있게 예측하기 위해서
class Predict_Captcha_Label :
  def __init__(self, model_path, bow) :
    self.MODEL = CRNN_Module.load_from_checkpoint(model_path, bow=bow)
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    self.TRAINER = pl.Trainer(accelerator=DEVICE, enable_progress_bar=False, logger=False)

  def __preprocessing_Image(self, image) :
    USER_IMAGE = Image.fromarray(np.uint8(image)).convert("L").convert("RGB")
    USER_IMAGE = Compose([
      ToTensor(),
      Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])(USER_IMAGE)
    return USER_IMAGE

  def __convert_Image_To_Data_Loader(self, images) :
    user_images = list(map(self.__preprocessing_Image, images))
    while len(user_images) != 8 :
      user_images.append(user_images[0])
    
    USER_IMAGES = torch.stack(user_images)
    USER_IMAGE_DATASET = TensorDataset(USER_IMAGES, torch.zeros(8))
    USER_IMAGE_LOADER = DataLoader(USER_IMAGE_DATASET, batch_size=8)
    return USER_IMAGE_LOADER
  
  # 단일 이미지에 대한 예측 결과를 반환시키기 위해서
  def predict_Captcha_Label(self, image) :
      DATA_LOADER = self.__convert_Image_To_Data_Loader([image])
      PRED = self.TRAINER.predict(self.MODEL, dataloaders=DATA_LOADER)[0][0]
      return PRED
  
  # 여러 이미지에 대한 예측 결과를 반환시키기 위해서
  def predict_Captcha_Labels(self, images) :
    DATA_LOADER = self.__convert_Image_To_Data_Loader(images)
    PREDS = self.TRAINER.predict(self.MODEL, dataloaders=DATA_LOADER)[0][:len(images)]
    return PREDS

In [ ]:
import requests
import os

from PIL import Image
from io import BytesIO
import numpy as np

# HTTP 요청에 대한 통합된 인터페이스를 제공하기 위한 라이브러리
class Request :
  # 자연스러운 요청을 위한 기반 헤더
  GLOBAL_HEADERS = {"Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                    "Accept-Encoding" : "gzip, deflate, br",
                    "Accept-Language" : "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}

  # HTTP GET 요청을 수행하고 응답을 받기 위해서
  @staticmethod
  def get(url, headers={}, cookies={}) :
    headers.update(Request.GLOBAL_HEADERS)
    return requests.get(url, headers=headers, cookies=cookies)
  
  # HTTP POST 요청을 수행해고 응답을 받기 위해서
  @staticmethod
  def post(url, headers={}, cookies={}, data={}) :
    headers.update(Request.GLOBAL_HEADERS)
    return requests.post(url, headers=headers, cookies=cookies, data=data)

# DC 인사이드의 캡차 이미지를 얻기 위해서
class Dcincide_Captcha_API :
  def __init__(self) :
    self.GALL_ID = "lilyfever"
    self.KCAPTCHA_TYPE = "recommend"
    self.GALL_TYPE = "M"

    self.SESSION_COOKIES = Request.get(f"https://gall.dcinside.com/mgallery/board/write/?id={self.GALL_ID}").cookies
    self.SESSION_COOKIE_INFO = {"PHPSESSID":self.SESSION_COOKIES["PHPSESSID"], "ci_c":self.SESSION_COOKIES["ci_c"]}
    self._init_Captcha_Session()
  
  # 새로운 캡차 이미지를 얻기 위해서 캡차 세션을 초기화시킴
  def _init_Captcha_Session(self) :
    SESSION_HTTP_HEADER = {"Content-Type":"application/x-www-form-urlencoded; charset=UTF-8", "X-Requested-With":"XMLHttpRequest"}
    SESSION_DATAS = {"ci_t":self.SESSION_COOKIES["ci_c"], "gall_id":self.GALL_ID, "kcaptcha_type":self.KCAPTCHA_TYPE, "_GALLTYPE_":self.GALL_TYPE}
    Request.post("https://gall.dcinside.com/kcaptcha/session", SESSION_HTTP_HEADER, self.SESSION_COOKIE_INFO, SESSION_DATAS)

  # 생성된 캡차 세션에 존재하는 캡차 이미지를 PIL 형태로 얻기 위해서
  def _captcha_Image(self) :
    IMG_RES = Request.get(f"https://gall.dcinside.com/kcaptcha/image/?gall_id={self.GALL_ID}&kcaptcha_type={self.KCAPTCHA_TYPE}", {}, self.SESSION_COOKIE_INFO)
    return Image.open(BytesIO(IMG_RES.content))

  # 새로운 캡차 문자열 이미지를 얻기 위해서
  def new_Captcha_Image(self) :
    self._init_Captcha_Session()
    return self._captcha_Image()
  
  # 이전과 정답이 동일하면서 형태만 다른 캡차 문자열 이미지를 얻기 위해서
  def same_Captcha_Image(self) :
    return self._captcha_Image()

# 파일시스템 관련 조작을 위해서
class File_System :
  # 디렉토리가 없을 경우 생성시키기 위해서
  @staticmethod
  def make_Directory_If_Not_Exist(dir_path) :
    if not os.path.exists(dir_path) : os.mkdir(dir_path)
  
  # 디렉토리 내부의 파일 개수를 반환받기 위해서
  @staticmethod
  def count_Num_Of_Files(dir_path) :
    return len(os.listdir(dir_path))

  # 특정 디렉토리를 .zip 형태로 압축시키기 위해서
  @staticmethod
  def compress_With_Zip(dir_path, zip_name) :
    os.system(f"zip -r {zip_name} {dir_path}")

  # 특정 디렉토리를 완전하게 삭제시키기 위해서
  @staticmethod
  def remove_Directory(dir_path) :
    os.system(f"rm -r {dir_path}")

# 자료형변환을 간편하게 수행하기 위한 라이브러리
class Convert :
  # 넘파이 배열을 PIL 이미지 형태로 변환시키기 위해서
  @staticmethod
  def numpy_Array_To_PIL_Image(numpy_array, color_channel="RGB") :
    return Image.fromarray(np.uint8(numpy_array)).convert(color_channel)

In [ ]:
import time

# 캡차 이미지를 자동으로 생성시키기 위해서
IMAGE_DIRECTORY = "./captchas" # 출력시킬 캡차 폴더명
File_System.make_Directory_If_Not_Exist(IMAGE_DIRECTORY)

CAPTCHA_API = Dcincide_Captcha_API()
PREDICT_CAPTCHA_LABEL = Predict_Captcha_Label("/content/Resnet_With_Attention_Complete_Train-0.9992_Valid-0.9841_Test-0.9870", bow=TRAIN_DATASET.BOW)

target_count = 10000 # 다운로드시킬 캡차 이미지의 총 개수
while True :
  captcha_images = []
  save_image = None
  while True :
    try :
      captcha_images = []
      captcha_images.append(CAPTCHA_API.new_Captcha_Image())
      captcha_images.append(CAPTCHA_API.same_Captcha_Image())
      captcha_images.append(CAPTCHA_API.same_Captcha_Image())
      save_image = CAPTCHA_API.same_Captcha_Image()
      break
    except :
      time.sleep(3.0)
      CAPTCHA_API = Dcincide_Captcha_API()
  
  TOTAL_FILE_COUNT = File_System.count_Num_Of_Files(IMAGE_DIRECTORY)
  PRED_LABELS = PREDICT_CAPTCHA_LABEL.predict_Captcha_Labels(captcha_images)
  if PRED_LABELS.count(PRED_LABELS[0]) == len(PRED_LABELS) :
    IMAGE_FILE_PATH_TO_SAVE = f"{IMAGE_DIRECTORY}/{PRED_LABELS[0]}.jpg"
    save_image.save(IMAGE_FILE_PATH_TO_SAVE)
    print(f"[*] 예측 일치로 인한 이미지 추가 ! : 파일 총 개수 - {TOTAL_FILE_COUNT+1}/{target_count} / 대상 이미지 위치 : {IMAGE_FILE_PATH_TO_SAVE}")
    if TOTAL_FILE_COUNT + 1 >= target_count : break
    
  else :
    print(f"[*] 예측 불일치로 이미지 추가 실패 ! : 파일 총 개수 - {TOTAL_FILE_COUNT}/{target_count}")

File_System.compress_With_Zip(IMAGE_DIRECTORY, "./" + IMAGE_DIRECTORY.split("/")[-1] + ".zip")
print("[*] 캡차 자동 추가 시스템 작동 완료 !")